# 1. Importando os dados:

Abaixo, usamos pandas para importar os dados, mas logo transformamos o DataFrame em uma lista, para tentar nos adequar ao máximo ao conteúdo já ministrado.

In [48]:
import pandas as pd

dados = pd.read_json('brasileirao-2019.json')


campeonato = []

for i in dados.columns:
  campeonato.append(list(dados[i]))

# 2. Funções usadas:
Abaixo, criamos as funções que irão ser usadas ao longo do projeto:

In [42]:
def tecnicos(campeonato: list) -> list:
  """retorna a lista de técnicos que participaram do campeonato"""
  tec = sorted(
      list(
          set(
              [
                  tecnico
                  for rodada in campeonato
                  for partida in rodada
                  for tecnico in partida['coach'].values()
              ]
            )
          )
      )
  return tec



def times(campeonato: list) -> list:
  """retorna uma lista com os times que disputaram o campeonato em ordem alfabética"""
  t = sorted(
      list(
          set(
              [
                  time
                  for rodada in campeonato
                  for partida in rodada
                  for time in partida['clubs'].values()
              ]
            )
          )
      )
  return t



def times_por_tecnicos(tecnico: str, campeonato: list) -> dict:
  """Retorna um dicionário, as chaves são times,
  e os valores a quantidade de partidas que o técnico esteve no time"""
  times = [
    partida['clubs'][chave]
    for rodada in campeonato
    for partida in rodada
    for chave in ['home', 'away']
    if partida['coach'][chave] == tecnico
]
  return {time:times.count(time) for time in times}


def chave_tecnico(tecnico: str, partida: dict) -> str:
  """Recebe um tecnico e uma partida, e retorna a chave
  que indica se o tecnico jogou em casa ou fora de casa"""
  if tecnico not in partida['coach'].values():
    return ""
  return [ch for ch in partida['coach'].keys() if partida['coach'][ch] == tecnico][0]



def chave_time(time: str, partida: dict) -> str:
  """Recebe um time e uma partida, e retorna a chave
  que indica se o time jogou em casa ou fora de casa"""
  if time not in partida['clubs'].values():
    return ""
  return [ch for ch in partida['clubs'].keys() if partida['clubs'][ch] == time][0]



def tecnico_de(time: str, partida: dict) -> str:
  """Retorna o ténico do time na partida"""
  return "" if time not in partida['clubs'].values() else partida['coach'][chave_time(time, partida)]



def time_de(tecnico: str, partida: dict) -> str:
    """retorna o time treinado por um tecnico em uma partida"""
    return "" if tecnico not in partida['coach'].values() else partida['clubs'][chave_tecnico(tecnico, partida)]



def vencedor(partida: dict) -> str:
  """recebe uma partida e retorna o time vencedor"""
  if partida['goals']['home'] == partida['goals']['away']:
    return "empate"
  else:
    home = int(partida['goals']['home'])
    away = int(partida['goals']['away'])
    return partida['clubs']['home'] if home > away else partida['clubs']['away']



def pontos(time: str, campeonato: list) -> int:
  """retorna a quantidade de pontos de um time em um campeonato"""
  pontos = 0
  partidas = [
      partida
      for rodada in campeonato
      for partida in rodada
      if time in partida['clubs'].values()
    ]
  for partida in partidas:
    if vencedor(partida) == time:
      pontos += 3
    elif vencedor(partida) == 'empate':
      pontos += 1
  return pontos



def pontos_tecnico(tecnico: str, time: str, campeonato: list) -> int:
    """Retorna a quantidade de pontos que um técnico conseguiu para um dado time em um campeonato"""
    pontos = 0
    for rodada in campeonato:
        for partida in rodada:
            if time in partida['clubs'].values() and tecnico_de(time, partida) == tecnico:
                if vencedor(partida) == time:
                    pontos += 3
                elif vencedor(partida) == 'empate':
                    pontos += 1
    return pontos

# 3. Pontos por Partida. 

Para tentar mensurar a eficiência de um técnico em um time, vamos calcular a razão entre a quantidade de pontos que o time obteve sob o comando do técnico e a quantidade de partidas disputadas pelo time sob o comando do mesmo técnico. 

In [49]:
pontos_tecnico("R. Ceni", "Fortaleza", campeonato)

45

In [50]:
times_por_tecnicos("R. Ceni", campeonato)

{'Fortaleza': 29, 'Cruzeiro': 7}

In [57]:
tecs = tecnicos(campeonato)

eficiencia = {tec:
     {
         time:
         pontos_tecnico(tec, time, campeonato)/(times_por_tecnicos(tec, campeonato)[time])
         for time in times_por_tecnicos(tec, campeonato).keys()
     } 
     for tec in tecs}

a

{'A. Batista': {'Ceará SC': 1.0769230769230769, 'Cruzeiro': 0.0},
 'A. Fucks': {'CSA': 1.0, 'Ceará SC': 0.6666666666666666},
 'A. Lopes dos Santos': {'Palmeiras': 3.0},
 'A. Stival': {'São Paulo': 1.6666666666666667},
 'A. Valentim do Carmo Neto': {'Avaí': 0.8666666666666667, 'Botafogo': 1.0},
 'A. da Silva Braga': {'Flamengo': 1.6666666666666667,
  'Cruzeiro': 1.2142857142857142},
 'B. Lazaroni': {'Botafogo': 1.5},
 'C. A. P. F. Hembert': {'Fortaleza': 0.0},
 'C. dos Santos Oliveira': {'Goiás': 1.4166666666666667},
 'Coelho': {'Corinthians': 1.375},
 'E. Alves Moreira': {'Ceará SC': 1.0454545454545454},
 'E. Hartkopp': {'Chapecoense': 0.75},
 'E. Machado Souto': {'Avaí': 0.4444444444444444},
 'E. Spinassé Camillato': {'Avaí': 0.21428571428571427},
 'E. de Barros': {'Athletico-PR': 2.25},
 'E. de Souza Barroca': {'Botafogo': 1.173913043478261},
 'F. Carille': {'Corinthians': 1.5},
 'F. Diniz Silva': {'Fluminense': 0.8, 'São Paulo': 1.5625},
 'G. Santos Vasconcelo': {'CSA': 0.0},
 'J. M

In [82]:
a = {tec:times_por_tecnicos(tec, campeonato) for tec in tecs}

b = {tec:{time:a[tec][time] for time in a[tec].keys() if a[tec][time]==max(a[tec].values())} for tec in a.keys()}

c = {tec:list(b[tec].items())[0][1] for tec in b.keys()}

d = dict(sorted(c.items(), key=lambda item: item[1]))

e = {tec:b[tec] for tec in d}


# dicionário
# chaves sendo técnicos
# valores sendo dicionários com times em que mais atuaram no campeonato e longevidade 
# ordenados por longevidade
e

{'C. A. P. F. Hembert': {'Fortaleza': 1},
 'J. Pires de Deus': {'Flamengo': 1},
 'M. Longo de Araújo': {'São Paulo': 1},
 'M. Montenegro': {'Fortaleza': 1},
 'R. Alves Resende': {'Cruzeiro': 1},
 'R. L. Martins Gomes': {'Goiás': 1},
 'A. Lopes dos Santos': {'Palmeiras': 2},
 'B. Lazaroni': {'Botafogo': 2},
 'G. Santos Vasconcelo': {'CSA': 3},
 'M. Salles': {'Flamengo': 3},
 'R. Colbachini': {'Internacional': 3},
 'M. Gomes Valadares': {'Vasco da Gama': 4},
 'O. de Oliveira Filho': {'Fluminense': 6},
 'Coelho': {'Corinthians': 8},
 'E. Hartkopp': {'Chapecoense': 8},
 'E. de Barros': {'Athletico-PR': 8},
 'E. Machado Souto': {'Avaí': 9},
 'M. Ribeiro Cabo': {'CSA': 9},
 'J. Mannarino': {'Internacional': 11},
 'C. dos Santos Oliveira': {'Goiás': 12},
 'A. Batista': {'Ceará SC': 13},
 'V. Carmo Mancini': {'Atlético-MG': 13},
 'A. da Silva Braga': {'Cruzeiro': 14},
 'E. Spinassé Camillato': {'Avaí': 14},
 'A. Valentim do Carmo Neto': {'Avaí': 15},
 'F. Diniz Silva': {'São Paulo': 16},
 'L. 